In [1]:
from sympy import *
import numpy as np
import scipy.constants as c

from tabulate import tabulate

import pandas as pd
import urllib.request

def lc_read_csv(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"
lc_iso = livechart + "fields=ground_states"

def iso_molar(iso): #accepts string of Z number followed by atomic symbol
    iso_info = lc_read_csv(lc_iso + f"&nuclides={iso}")
    mu_iso_molar = iso_info["atomic_mass"][0]
    iso_molar = mu_iso_molar/10**6
    return iso_molar #returns molar mass in amu

def u_mev(u):
    MeV = u * (9.31494028*100)
    return MeV

In [20]:
#Conversions

N_A = c.Avogadro
y_to_s = 364.25 * 24 * 60 * 60
Ci_to_bq = 37000000000

In [ ]:
#1, Yes Jupyter

In [64]:
#2

mass_UO2 = 1 * 1000 * .7 #kg * g/kg * UO2%
mass_PuO2 = 1 * 1000 * .3 #kg * g/kg * PuO2%

dc_U238 = log(2) / (4.463 * 1e9 * y_to_s) # ga * a/ga * s/a
dc_Pu239 = log(2) / (24.11 * 1e3 * y_to_s) # ka * a/ka * s/a
dc_Pu240 = log(2) / (6.561 * 1e3 * y_to_s) # ka * a/ka * s/a
dc_Pu241 = log(2) / (14.329 * y_to_s) # a * s/a
dc_Pu242 = log(2) / (375 * 1e3 * y_to_s) # ka * a/ka * s/a

m_U238O2 = mass_UO2 * 1
m_Pu239O2 = mass_PuO2 * .705
m_Pu240O2 = mass_PuO2 * .213
m_Pu241O2 = mass_PuO2 * .055
m_Pu242O2 = mass_PuO2 * .027

M_U238O2 = iso_molar('U238') + 2*iso_molar('O16')
M_Pu239O2 = iso_molar('Pu239') + 2*iso_molar('O16')
M_Pu240O2 = iso_molar('Pu240') + 2*iso_molar('O16')
M_Pu241O2 = iso_molar('Pu241') + 2*iso_molar('O16')
M_Pu242O2 = iso_molar('Pu242') + 2*iso_molar('O16')

n_U238 = m_U238O2 / M_U238O2 * N_A # g / g/mol * atoms/mol
n_Pu239 = m_Pu239O2 / M_Pu239O2 * N_A # g / g/mol * atoms/mol
n_Pu240 = m_Pu240O2 / M_Pu240O2 * N_A # g / g/mol * atoms/mol
n_Pu241 = m_Pu241O2 / M_Pu241O2 * N_A # g / g/mol * atoms/mol
n_Pu242 = m_Pu242O2 / M_Pu242O2 * N_A # g / g/mol * atoms/mol

a_U238 = n_U238 * dc_U238
a_Pu239 = n_Pu239 * dc_Pu239
a_Pu240 = n_Pu240 * dc_Pu240
a_Pu241 = n_Pu241 * dc_Pu241
a_Pu242 = n_Pu242 * dc_Pu242

a_total = (a_U238 + a_Pu239 + a_Pu240 + a_Pu241 + a_Pu242).evalf()

print('Activity [Tbq]', a_total/1e12)
print('Activity [kCi]', a_total / Ci_to_bq /1e3)

Activity [Tbq] 56.8415436859535
Activity [kCi] 1.53625793745820


In [47]:
#3

dc_K40 = (log(2) / 1.248e9).evalf()

N_K40_10e5 = exp(-dc_K40 * 1e5)
N_K40_10e7 = exp(-dc_K40 * 1e7)
N_K40_10e9 = exp(-dc_K40 * 1e9)

N_Ar40_10e5 = .107 * (1 - N_K40_10e5)
N_Ar40_10e7 = .107 * (1 - N_K40_10e7)
N_Ar40_10e9 = .107 * (1 - N_K40_10e9)

print('Ratio after 10e5 years;', N_Ar40_10e5 / N_K40_10e5)
print('Ratio after 10e7 years;', N_Ar40_10e7 / N_K40_10e7)
print('Ratio after 10e9 years;', N_Ar40_10e9 / N_K40_10e9)

Ratio after 10e5 years; 5.94301346092796e-6
Ratio after 10e7 years; 0.000595938249942017
Ratio after 10e9 years; 0.0794634477281412


#4a

6Li(n,t)4He
16O(t,n)18F

In [78]:
#4b
q_1 = u_mev((iso_molar('6Li') + 1.0086649159) - (iso_molar('3H') + iso_molar('4He')))
print('Q 6Li(n,t)4He: [MeV]', q_1)

q_2 = u_mev((iso_molar('16O') + iso_molar('3H')) - (iso_molar('18F') + 1.0086649159))
print('Q 16O(t,n)18F: [MeV]', q_2)

Q 6Li(n,t)4He: [MeV] 4.783471353085346
Q 16O(t,n)18F: [MeV] 1.2683787915797184


#4c

both Q>0, so there is no threshold.

#4d

Yes

In [53]:
#5a

alpha_5a = (16 - 1)**2 / (16 + 1)**2
xi_5a = 1 + alpha_5a / (1-alpha_5a) * log(alpha_5a)

n_5a = 1/xi_5a * log(2/1)
print('Scatters 16O;', n_5a)

Scatters 16O; 5.77879560768414


In [54]:
#5b

alpha_5b = (56 - 1)**2 / (56 + 1)**2
xi_5b = 1 + alpha_5b / (1-alpha_5b) * log(alpha_5b)

n_5b = 1/xi_5b * log(2/1)
print('Scatters 56Fe;', n_5b)

Scatters 56Fe; 19.6398709003390


In [57]:
#7a

mu = (log(10) / 2.8).evalf()
print('Linear Attenuation Coefficient [1/cm]', mu)

Linear Attenuation Coefficient [1/cm] 0.822351818926445


In [59]:
#7b

half_thickness = (log(2) / mu).evalf()
print('Half Thickness [cm]', half_thickness)

Half Thickness [cm] 0.842883987859147


In [66]:
#7c
print('MFP;', 1/mu)

MFP; 1.21602454932911


In [63]:
#8a
(.720/100 * 700) + (.0055/100 * 116) + ((100-.72-.0055)/100 * 12.2)

17.157868999999998

In [62]:
#8b
(.720/100 * 587) + (.0055/100 * .465) + ((100-.72-.0055)/100 * 11.8e-6)

4.226437289391001